In [ ]:
from dask_sql import Context
from dask.datasets import timeseries
from dask.distributed import Client

In [ ]:
# Create a context to hold the registered tables
c = Context()

In [ ]:
client = Client()
client

In [ ]:
# Load the data and register it in the context
# This will give the table a name
df = timeseries()
c.register_dask_table(df, "timeseries")

In [ ]:
# Now execute an SQL query. The result is a dask dataframe
# The query looks for the id with the highest x for each name
# (this is just random test data, but you could think of looking
# for outliers in the sensor data)
result = c.sql("""
    SELECT
        lhs.name,
        lhs.id,
        lhs.x
    FROM
        timeseries AS lhs
    JOIN
        (
            SELECT
                name AS max_name,
                MAX(x) AS max_x
            FROM timeseries
            GROUP BY name
        ) AS rhs
    ON
        lhs.name = rhs.max_name AND
        lhs.x = rhs.max_x
""")

In [ ]:
# Show the result...
print(result.compute())

In [ ]:
# ... or use it for any other dask calculation
# (just an example, could also be done via SQL)
print(result.x.mean().compute())